In [1]:
!pip install -q pdfminer.six pytesseract pdf2image sentence-transformers pinecone-client groq langchain

# Install Poppler for PDF to image conversion
!apt-get install -y poppler-utils

# Install Tesseract OCR
!apt-get install -y tesseract-ocr

# Download language data for Tesseract (optional, English is default)
!apt-get install -y tesseract-ocr-eng


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.4 MB/s eta 0:00:00
Reading package lists... Done
Building depend

In [3]:
# Uninstall the old package that causes conflict
!pip uninstall -y pinecone-client

# Install the correct official Pinecone package
!pip install pinecone


Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 9.7 MB/s eta 0:00:00


In [5]:
import os
from pinecone import Pinecone, ServerlessSpec
from groq import Groq
from PIL import Image
import pytesseract
from pdfminer.high_level import extract_text
from pdf2image import convert_from_path
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 🔐 Set your keys here
os.environ["GROQ_API_KEY"] = "your-groq-api-key"
pinecone_api_key = "your-real-pinecone-api-key"
pinecone_index_name = "chatbot-index"
pinecone_host = "chatbot-index-abc123.svc.us-east1-gcp.pinecone.io"


# Initialize Groq and Pinecone
groq_client = Groq(api_key=os.environ["GROQ_API_KEY"])
pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index(name=pinecone_index_name, host=pinecone_host)

embedder = SentenceTransformer("sentence-transformers/all-roberta-large-v1")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [7]:
!pip install pdfminer.six pytesseract pdf2image
!apt-get install -y poppler-utils tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [8]:
from pdfminer.high_level import extract_text
from pdf2image import convert_from_path
from PIL import Image
import pytesseract

def extract_pdf_text(path):
    """
    Extracts text from a PDF file.
    - First tries pdfminer (text-based PDFs)
    - Falls back to pytesseract OCR (scanned/image PDFs)
    """
    try:
        # Try text extraction from text-based PDF
        text = extract_text(path)
        if len(text.strip()) > 50:
            return text
    except Exception as e:
        print("pdfminer failed:", e)

    print("Using OCR fallback with pytesseract...")
    # Convert PDF pages to images
    images = convert_from_path(path)
    ocr_text = ""
    for img in images:
        ocr_text += pytesseract.image_to_string(img)
    return ocr_text


In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_text(text)


In [11]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("sentence-transformers/all-roberta-large-v1")

def embed_chunks(chunks):
    return [embedder.encode(chunk).tolist() for chunk in chunks]


In [13]:
import uuid

def store_chunks_in_pinecone(chunks):
    embeddings = [embedder.encode(chunk).tolist() for chunk in chunks]
    vectors = [
        {
            "id": str(uuid.uuid4()),
            "values": vec,
            "metadata": {"text": chunk}
        }
        for chunk, vec in zip(chunks, embeddings)
    ]
    index.upsert(vectors=vectors)


In [14]:
def retrieve_context(query, top_k=5):
    query_vec = embedder.encode(query).tolist()
    results = index.query(vector=query_vec, top_k=top_k, include_metadata=True)
    return "\n".join([match["metadata"]["text"] for match in results["matches"]])


In [15]:
def ask_groq(context, query):
    prompt = f"""
You are a helpful assistant. Use the following context to answer the question.

Context:
{context}

Question: {query}
Answer:
"""
    response = groq_client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()


In [16]:
def chatbot_pipeline(pdf_path, query):
    text = extract_pdf_text(pdf_path)
    chunks = chunk_text(text)
    store_chunks_in_pinecone(chunks)
    context = retrieve_context(query)
    answer = ask_groq(context, query)
    print("🤖 Answer:", answer)


In [24]:
from google.colab import files

# Upload the PDF file
uploaded = files.upload()


Saving quiz_22l6953.pdf to quiz_22l6953.pdf


In [30]:
from google.colab import files

# Upload the PDF file
uploaded = files.upload()


Saving Optimization_of_the_Functional_Decomposition_of_Parallel_and_Distributed_Computations_in_Graph_Coloring_With_the_Use_of_High-Performance_Computing-1.pdf to Optimization_of_the_Functional_Decomposition_of_Parallel_and_Distributed_Computations_in_Graph_Coloring_With_the_Use_of_High-Performance_Computing-1.pdf


In [31]:
from PIL import Image
import pytesseract

# Convert the PDF to images (one image per page)
images = convert_from_path('/content/Optimization_of_the_Functional_Decomposition_of_Parallel_and_Distributed_Computations_in_Graph_Coloring_With_the_Use_of_High-Performance_Computing-1.pdf')

# Perform OCR on the first page (adjust for other pages if necessary)
text_from_ocr = pytesseract.image_to_string(images[0])

# Print OCR text
print(text_from_ocr[:500])  # Print the first 500 characters


IEEE Access

"Multdscpinary : Rapid Review £ Open Access Journal

 

Received March 8, 2022, accepted March 21, 2022, date of publication March 25, 2022, date of current version April 5, 2022.

Digital Object Identifier 10.1109/ACCESS.2022.3162215

Optimization of the Functional Decomposition
of Parallel and Distributed Computations
in Graph Coloring With the Use of

High-Performance Computing

JARMILA SKRINAROVA® AND ADAM DUDAS®

Department of Computer Science, Faculty of Natural Sciences, Mate


In [29]:
from pdfminer.high_level import extract_text

# Specify the correct path for the uploaded PDF file
pdf_path = '/content/quiz_22l6953.pdf'

# Extract the text from the PDF
text = extract_text(pdf_path)

# Check if text was extracted
if text:
    print("Text extracted successfully!")
    print(text[:500])  # Print the first 500 characters of extracted text
else:
    print("No text extracted.")


Text extracted successfully!
Q1: Explain the purpose of a Dropout layer in CNNs. Discuss how it works during training vs 
inference. 

The purpose of a Dropout layer is to prevent overfitting in neural networks by introducing 
regularization. It works by randomly setting a fraction of input units to zero during training, 
thereby preventing units from co-adapting too much. 

●  During training: A specified dropout rate  means that each neuron has a 50% chance of 

being set to zero at each update cycle. 

●  During inferenc
